<a href="https://colab.research.google.com/github/AlexKressner/Industrielles_Management/blob/main/Produktionsplanung_Erweiterung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Produktionsplanung
## Capacitated Lot Sizing Problem (CLSP)

## Modell

### Indexmengen
$p \in P$ : Menge der Produkte

$t \in T$ : Menge der Planungsperioden (Wochen)



### Parameter
$cap_t$ : Kapazität der Abfüllanlage in Woche $t$

$lc_{p}$ : Lagerkosten für Produkt $p$

$rc_{p}$ : Rüstkosten für Produkt $k$

$bz_p$ : Bearbeitungszeit für Produkt $p$

$rz_{pp'}$ : Zeit für Umrüsten von Produkt $p$ auf Produkt $p'$

$d_{pt}$ : Nachfrage nach Produkt $p$ in Woche $t$

$s_{p}$ : Anfänglicher Rüstzustand

### Variablen
$Q_{pt} \ge 0$ : Produktionsmenge von Produkt $p$ in Woche $t$

$B_{pt} \ge 0$ : Lagerbestand von Produkt $p$ in Woche $t$

$R_{pp't} \in \{0,1\}$ : Binäre Rüstvariable

$S_{pt} \in \{0,1\}$ : Rüstzustandsvariable

### Zielfunktion
Min $K = \sum_{p,p',t} rc_{p'} * R_{p,p',t} + \sum_{p,t} lc_p * B_{pt}$

### Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

**(2) Produktionskapazitäten**

$\sum_{p} bz_p * Q_{pt} + \sum_{p,p'} rz_{pp'} * R_{pp't} \le pcap_{t}$

$∀ t$

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * S_{pt} $

$∀ p, t$

**(4) Koppelung Rüstentscheidung mit Rüstzustand**

$R_{pp't} \ge S_{p't} + S_{p,t-1} -1 $

$∀ p,p', t$

**(5) Eindeutiger Rüstzustand**

$\sum_{p} S_{pt} = 1 $

$∀ t$

## Implementierung

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Datenaufbereitung


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ordner finden
! ls drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung

Erweiterung_Nachfrage.csv   Kosten_Zeiten.csv
Erweiterung_Rüstzeiten.csv  Nachfrage.csv


In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung && pwd

/content/drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung


In [ ]:
# Daten laden
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/Industrielles_Management/Daten/Produktionsplanung"

In [ ]:
# Nachfrage
nachfrage = pd.read_csv(f"{path}/Erweiterung_Nachfrage.csv", sep=";")

In [ ]:
nachfrage.head()

,Produkt,Tag,Nachfrage
0,Well,1,0
1,Well,2,0
2,Well,3,0
3,Well,4,0
4,Well,5,0


In [ ]:
# Kosten & Zeiten
kosten_zeiten = pd.read_csv(f"{path}/Kosten_Zeiten.csv", sep=";", decimal=",")

In [ ]:
kosten_zeiten

,Produkt,Rüstkosten,Lagerkosten,Rüstzeit,Bearbeitungszeit
0,Well,100,3,420,1
1,BraTee,250,5,700,2
2,HafTee,200,6,560,1
3,DirTee,150,4,560,3
4,StraB,250,3,1400,2


In [ ]:
# Reihenfolgeabhängige Rüstzeiten
rüstzeiten = pd.read_csv(f"{path}/Erweiterung_Rüstzeiten.csv", sep=";", decimal=",")

In [ ]:
rüstzeiten.head()

,Produkt_Von,Produkt_Zu,Rüstzeit
0,Well,Well,0
1,Well,BraTee,42
2,Well,HafTee,42
3,Well,DirTee,42
4,Well,StraB,35


In [ ]:
s = {}
s["Well"] = 1

## Indexmengen

In [ ]:
P = nachfrage["Produkt"].unique().tolist() # Menge der Produkte

In [ ]:
T = nachfrage["Tag"].unique().tolist() # Menge der Planungsperioden (Tage)

## Entscheidungsvariablen

In [ ]:
infinity = solver.infinity()

In [ ]:
# Produktionsmengen
Q={}
for p in P:
  for t in T:
    Q[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Lagerbestand
B={}
for p in P:
  for t in T:
    B[p,t] = solver.NumVar(0.0, infinity, f"{p},{t}")

In [ ]:
# Rüstvariablen
R={}
for p in P:
  for pp in P:
    for t in T:
      R[p,pp,t] = solver.BoolVar(f"{p},{pp},{t}")

In [ ]:
# Rüstzustand
S={}
for p in P:
  for t in T:
    S[p,t] = solver.BoolVar(f"{p},{t}")

In [ ]:
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 1200


## Parameter

In [ ]:
# Kosten
lc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Lagerkosten"]
rc = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Rüstkosten"]

In [ ]:
# Zeiten
bz = kosten_zeiten.set_index(["Produkt"]).to_dict("dict")["Bearbeitungszeit"]
rz = rüstzeiten.set_index(["Produkt_Von","Produkt_Zu"]).to_dict("dict")["Rüstzeit"]

In [ ]:
# Nachfrage
d = nachfrage.set_index(["Produkt","Tag"]).to_dict("dict")["Nachfrage"]

In [ ]:
# Kapazitäten
cap = {}
for t in T:
  cap[t] = 960

## Zielfunktion

### Zielfunktion
Min $K = \sum_{p,p',t} rc_{p'} * R_{p,p',t} + \sum_{p,t} lc_p * B_{pt}$

In [ ]:
# Minimierung der gesamten Kosten
solver.Minimize(sum(rc[pp] * R[p,pp,t] for p in P for pp in P for t in T) + sum(lc[p] * B[p,t] for p in P for t in T))

## Nebenbedingungen

**(1) Lagerbilanzgleichung**

$B_{pt} = B_{p,t-1} + Q_{pt} - d_{pt}$

$∀ p, t$

In [ ]:
for p in P:
  for t in T:
    solver.Add(B[p,t] == B.get((p,t-1),0) + Q[p,t] - d[p,t])

**(2) Produktionskapazitäten**

$\sum_{p} bz_p * Q_{pt} + \sum_{p,p'} rz_{pp'} * R_{pp't} \le pcap_{t}$

$∀ t$

In [ ]:
for t in T:
  solver.Add(sum(bz[p] * Q[p,t] for p in P) + sum(rz[p,pp] * R[p,pp,t] for p in P for pp in P) <= cap[t])

**(3) Koppelung Produktions- mit Rüstentscheidung**

$Q_{pt} \le \frac{cap_t}{bz_p} * S_{pt} $

$∀ p, t$

In [ ]:
for p in P:
  for t in T:
    solver.Add(Q[p,t] <= (cap[t]/bz[p]) * S[p,t])

**(4) Koppelung Rüstentscheidung mit Rüstzustand**

$R_{pp't} \ge S_{p't} + S_{p,t-1} -1 $

$∀ p,p', t$

In [ ]:
for p in P:
  for pp in P:
    for t in T:
      if t > 1:
        solver.Add(R[p,pp,t] >= S[pp,t] + S[p,t-1] - 1)
      else:
        solver.Add(R[p,pp,t] >= S[pp,t] + s.get(p,0) - 1)

**(5) Eindeutiger Rüstzustand**

$\sum_{p} S_{pt} = 1 $

$∀ t$

In [ ]:
for t in T:
  solver.Add(sum(S[p,t] for p in P) == 1)

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Kosten) =', solver.Objective().Value())
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Kosten) = 63980.999999999985


In [ ]:
for t in T:
  print(f"Tag: {t}")
  produzieren = sum(bz[p] * Q[p,t].solution_value() for p in P)
  rüsten = sum(rz[p,pp] * R[p,pp,t].solution_value() for p in P for pp in P)
  print(f"Produktionszeit: {round(produzieren)}")
  print(f"Rüstzeit: {round(rüsten)}")
  print(f"Summe: {round(rüsten+produzieren)}")

  print("\n")


Tag: 1
Produktionszeit: 0
Rüstzeit: 0
Summe: 0


Tag: 2
Produktionszeit: 0
Rüstzeit: 0
Summe: 0


Tag: 3
Produktionszeit: 0
Rüstzeit: 0
Summe: 0


Tag: 4
Produktionszeit: 140
Rüstzeit: 0
Summe: 140


Tag: 5
Produktionszeit: 70
Rüstzeit: 42
Summe: 112


Tag: 6
Produktionszeit: 560
Rüstzeit: 25
Summe: 585


Tag: 7
Produktionszeit: 0
Rüstzeit: 215
Summe: 215


Tag: 8
Produktionszeit: 140
Rüstzeit: 42
Summe: 182


Tag: 9
Produktionszeit: 350
Rüstzeit: 70
Summe: 420


Tag: 10
Produktionszeit: 720
Rüstzeit: 42
Summe: 762


Tag: 11
Produktionszeit: 960
Rüstzeit: 0
Summe: 960


Tag: 12
Produktionszeit: 630
Rüstzeit: 30
Summe: 660


Tag: 13
Produktionszeit: 140
Rüstzeit: 25
Summe: 165


Tag: 14
Produktionszeit: 202
Rüstzeit: 185
Summe: 387


Tag: 15
Produktionszeit: 840
Rüstzeit: 30
Summe: 870


Tag: 16
Produktionszeit: 420
Rüstzeit: 70
Summe: 490


Tag: 17
Produktionszeit: 918
Rüstzeit: 42
Summe: 960


Tag: 18
Produktionszeit: 420
Rüstzeit: 30
Summe: 450


Tag: 19
Produktionszeit: 222
Rüstzeit